In [1]:
"""
Stock problem (Table 2)
"""

'\nStock problem (Table 2)\n'

In [2]:
#Import packages to facilitate this research
import os, random
SEED = 4                             
os.environ["PYTHONHASHSEED"]  = str(SEED) 
os.environ["TF_DETERMINISTIC_OPS"] = "1" 
random.seed(SEED)
import numpy as np 
np.random.seed(SEED) 
import tensorflow as tf #(https://www.tensorflow.org/install)
tf.keras.utils.set_random_seed(SEED)      
tf.config.experimental.enable_op_determinism() 
from tensorflow.keras import (layers,Model) 
from tensorflow.keras.layers import (Layer, Input, Dense, Concatenate, Activation, TimeDistributed,
Flatten, Lambda, GlobalAveragePooling1D, LeakyReLU) 
from tensorflow.keras.utils import to_categorical 
from concretedropout.tensorflow import (ConcreteDenseDropout, ConcreteSpatialDropout1D, # https://github.com/aurelio-amerio/ConcreteDropout
get_weight_regularizer, get_dropout_regularizer) 
from sktime.classification.deep_learning import InceptionTimeClassifier #https://www.sktime.net/en/stable/get_started.html
from sktime.classification.ensemble import BaggingClassifier
from aeon.classification.convolution_based import MultiRocketHydraClassifier #https://www.aeon-toolkit.org/en/stable/installation.html
from aeon.classification.early_classification import TEASER
from aeon.classification.hybrid import HIVECOTEV2
from aeon.datasets import load_classification 
from sklearn.metrics import (accuracy_score, mean_absolute_error, mean_squared_error,  # https://scikit-learn.org/stable/install.html
classification_report)
from sklearn.model_selection import train_test_split
from math import comb
from tqdm import tqdm
import time
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow.keras.callbacks import EarlyStopping
from statsmodels.stats.contingency_tables import mcnemar

/opt/anaconda3/lib/python3.12/site-packages/aeon/base/__init__.py:24: FutureWarning: The aeon package will soon be releasing v1.0.0 with the removal of legacy modules and interfaces such as BaseTransformer and BaseForecaster. This will contain breaking changes. See aeon-toolkit.org for more information. Set aeon.AEON_DEPRECATION_WARNING or the AEON_DEPRECATION_WARNING environmental variable to 'False' to disable this warning.
  warnings.warn(


In [3]:
#Load data 
file_path = '/Users/user/Downloads/earnings_latest.csv'
df = pd.read_csv(file_path)
file_path = '/Users/user/Downloads/stock_prices_latest.csv'
prices = pd.read_csv(file_path)
prices = prices.sort_values(by = [prices.columns[0], prices.columns[1]])
len(np.unique(prices.iloc[:,0]))
df = df.sample(frac = 1, random_state = 42).reset_index(drop = True)
df.iloc[:,1] = pd.to_datetime(df.iloc[:,1])
prices.iloc[:,1] = pd.to_datetime(prices.iloc[:,1])
time_series_list = []
before_earnings_list = []
on_and_after_earnings_list = []
ticker_prices_dict = {ticker:group for ticker, group in prices.groupby(prices.columns[0])}
valid_series_count = 0 
with tqdm(total = 168603   , desc = "Processing earnings dates") as pbar: 
    for index,row in df.iterrows(): 
        ticker = row[0]
        earnings_date = row[1]
        release_time = row.iloc[5]
        if pd.isna(release_time): 
            \
            continue 
        if release_time == "post": 
            earnings_date += pd.Timedelta(days = 1)
        elif release_time == "pre": 
            pass 
        else: 
            continue 
        if ticker in ticker_prices_dict: 
            ticker_prices = ticker_prices_dict[ticker]
            start_date_before = earnings_date - pd.Timedelta(days = 60 )
            before_earnings = ticker_prices[(ticker_prices.iloc[:,1] >= start_date_before) & (ticker_prices.iloc[:,1] < earnings_date)].iloc[:,6].values
            end_date_after = earnings_date + pd.Timedelta(days = 6)
            on_and_after_earnings = ticker_prices[(ticker_prices.iloc[:,1] >= earnings_date) & (ticker_prices.iloc[:,1] <= end_date_after)].iloc[:,6].values
            before_earnings_list.append(before_earnings)
            on_and_after_earnings_list.append(on_and_after_earnings)
            valid_series_count +=1
            pbar.update(1)
        if valid_series_count >= 168603 : 
            break
indices = [i for i in range(len(before_earnings_list))
             if len(before_earnings_list[i])     >= 40 and
                len(on_and_after_earnings_list[i]) > 0]
before_earnings_series = [before_earnings_list[i] for i in indices]
before_earnings_series = [series[-40:] for series in before_earnings_series]
on_and_after_earnings_series = [on_and_after_earnings_list[i] for i in indices]
on_earnings = []
for i in range(len(on_and_after_earnings_series)):
    if(len(on_and_after_earnings_series[i])>0):
        on_earnings.append(on_and_after_earnings_series[i][0])
before_earnings = np.array(before_earnings_series)
on_earnings = np.array(on_earnings)
before_earnings = np.expand_dims(before_earnings, axis = -1)
on_earnings = np.expand_dims(on_earnings, axis = -1) 
on_earnings = np.expand_dims(on_earnings, axis = -1) 
class_labels = []
for i in range(96614): 
    new = 0 
    new = on_earnings[i,0]
    old = 0 
    old = before_earnings[i,39,0]
    pc = 0 
    pc = ((new-old)/old) * 100 
    if pc>= 5 : 
        class_labels.append(1)
    elif pc <= -5: 
        class_labels.append(2)
    else: 
        class_labels.append(0)
print(class_labels.count(0))
print(class_labels.count(1))
print(class_labels.count(2))
class_labels = np.array(class_labels)

Processing earnings dates:  64%|████▍  | 107888/168603 [03:57<02:13, 454.06it/s]


61178
17802
17634


In [4]:
#Create approximately balanced problem 
idx_0 = np.where(class_labels == 0)[0]       
idx_1 = np.where(class_labels == 1)[0]       
idx_2 = np.where(class_labels == 2)[0]       
n_1 = len(idx_1)
print(f"class-1 count: {n_1}, class-0 count: {len(idx_0)}, class-2 count: {len(idx_2)}")
rng = np.random.default_rng(seed=42)         
idx_0_sampled = rng.choice(idx_0, size=n_1, replace=False)
balanced_idx = np.concatenate([idx_1, idx_2, idx_0_sampled])
balanced_idx = rng.permutation(balanced_idx)
before_earnings_bal = before_earnings[balanced_idx]   
on_earnings_bal     = on_earnings[balanced_idx]       
y_bal               = class_labels[balanced_idx]      
print(pd.Series(y_bal).value_counts())

class-1 count: 17802, class-0 count: 61178, class-2 count: 17634
1    17802
0    17802
2    17634
Name: count, dtype: int64


In [5]:
before_earnings = before_earnings_bal
on_earnings = on_earnings_bal
class_labels = y_bal

In [6]:
#create training and test sets 
balanced_idx = np.arange(len(class_labels))      
train_idx, test_idx = train_test_split(balanced_idx,test_size=0.20,random_state=42,stratify=class_labels[balanced_idx])
X_observed_train  = before_earnings[train_idx, :, :]
y_train           = class_labels[train_idx]
X_forecast_train  = on_earnings[train_idx, :, :]
X_observed_test   = before_earnings[test_idx, :, :]
y_test            = class_labels[test_idx]
X_forecast_test   = on_earnings[test_idx, :, :]  
y_train = to_categorical(y_train, num_classes=3)
y_test  = to_categorical(y_test,  num_classes=3)

In [7]:
def compute_returns_observed(X):
    """
    Function to compute the returns of the observed time series 
    """
    returns = (X[:, 1:, :] - X[:, :-1, :]) / X[:, :-1, :]
    return returns
X_observed_train_ret = compute_returns_observed(X_observed_train)
X_observed_test_ret = compute_returns_observed(X_observed_test)
def compute_returns_forecast(X_forecast, X_observed):
    """
    Function to compute the returns of the future time series 
    """
    last_price = X_observed[:, -1:, :]  #
    extended = np.concatenate([last_price, X_forecast], axis=1)
    returns = (extended[:, 1:, :] - extended[:, :-1, :]) / extended[:, :-1, :]
    return returns
X_forecast_train_ret = compute_returns_forecast(X_forecast_train, X_observed_train)
X_forecast_test_ret = compute_returns_forecast(X_forecast_test, X_observed_test)

In [8]:
X_observed_train = X_observed_train_ret
X_observed_test = X_observed_test_ret
X_forecast_train = X_forecast_train_ret
X_forecast_test = X_forecast_test_ret

In [9]:
print(X_observed_train.shape)
print(y_train.shape)
print(X_forecast_train.shape)
print(X_observed_test.shape)
print(y_test.shape)
print(X_forecast_test.shape)

(42590, 39, 1)
(42590, 3)
(42590, 1, 1)
(10648, 39, 1)
(10648, 3)
(10648, 1, 1)


In [10]:
Ns = X_observed_train.shape[0] #Number of time series 
time_steps = X_observed_train.shape[1] #Number of time steps per time series 
n_forecast = X_forecast_train.shape[1] #Number of steps ahead for the forecasts 
n_classes = y_train.shape[1] #Number of classes 
wr = get_weight_regularizer(Ns, l=1e-2, tau=0.5) #Determines the amount of regularization on the weights in the concrete dropout layers of ForeClassNet
dr = get_dropout_regularizer(Ns, tau=0.5, cross_entropy_loss=True) #Determines the amount of regularization on the dropout probabilities of ForeClassNet

In [11]:
def create_custom_filters(filter_length):
    """
    Function that takes in a user specified filter length to then construct hand-crafted filters 
    according to (4)-(6) in Section 5 of the research paper. 
    """
    filter_1 = np.array([(-1) ** (i + 1) for i in range(filter_length)], dtype=np.float32).reshape(filter_length, 1, 1) #Decreasing filter
    filter_2 = np.array([(-1) ** i for i in range(filter_length)], dtype=np.float32).reshape(filter_length, 1, 1) #Increasing filter
    filter_3 = np.zeros(filter_length) #Peaks filter
    for n in range(0, int(((filter_length - 3) / 3) + 1)): 
        filter_3[n] = (-3 / filter_length) * comb(int((filter_length - 3) / 3), n)
    for n in range(int(filter_length / 3), int((2 * (filter_length - 3) / 3) + 2)):
        filter_3[n] = (6 / filter_length) * comb(int((filter_length - 3) / 3), int(n - (filter_length / 3)))
    for n in range(int((2 * (filter_length - 3) / 3) + 2), filter_length):
        filter_3[n] = (-3 / filter_length) * comb(int((filter_length - 3) / 3), int(n - (2 * (filter_length - 3) / 3) - 2))
    filter_3 = filter_3.reshape(filter_length, 1, 1)
    custom_filters = np.concatenate([filter_1, filter_2, filter_3], axis=2) #Combine the filters together
    return custom_filters
class WelfordMeanVarianceLayer(Layer): #Defining the Welford Mean-variance layer (section 4.2) as a subclass of Layer in Keras
    def __init__(self, total_series, n_forecast=4, **kwargs): #Constructor of the class, where it takes in the number of time series we want to have mean and variance estimates for and the number of forecasts for each 
        super(WelfordMeanVarianceLayer, self).__init__(**kwargs) #Calls the Keras layer class
        self.total_series = total_series #Number of series the layer will track
        self.n_forecast = n_forecast #Number of forecast steps ahead
        self.stats = self.add_weight( #Creates a non-trainable tensor to store the means and variances which is initialized with zeros
            shape=(self.total_series, self.n_forecast, 2),
            initializer="zeros",
            trainable=False,
            name="stats"
        ) 
        self.count = self.add_weight( #Creates a non-trainable tensor to count how many samples for the mean and variance we have for the forecasts of a given time series, which are obtained through successive forward passes
            shape=(self.total_series,),
            initializer="zeros",
            trainable=False,
            name="count"
        )
    def reset_statistics(self): #A method to clear the existing estimates of the mean and variance of the forecasts and set them to zero, which you might want to do during testing
        self.stats.assign(tf.zeros_like(self.stats))
        self.count.assign(tf.zeros_like(self.count))
    def call(self, inputs, **kwargs): #Main method used when calling the Welford layer in a given forward pass
        forecast_values, series_ids = inputs  #Unpacks the inputs into the Welford layer, the forecast values in the current forward pass and the ids of the current time series being processed 
        batch_size = tf.shape(forecast_values)[0] #Derives the current batch size from the shape of the forecast values tensor
        def loop_body(i, current_stats, current_count): #Defines the body of a while loop to be used in the proceeding lines. Processes the ith time series in the batch by applying Welford's algorithm 
            idx = series_ids[i]   #Determines the id of the current time series being processed    
            val = forecast_values[i] #The values to be used to update the means and variances of the forecasts
            current_count = tf.tensor_scatter_nd_add(current_count, [[idx]], [1.0]) #Increments the count of the number of times we have updated the mean and variance of the given time series 
            old_mean = current_stats[idx, :, 0] #Extracts the old forecast means
            old_var  = current_stats[idx, :, 1] #Extracts the old forecast variance 
            n = current_count[idx] #Number of times we have updated the mean and variance of the forecasts for a given time series
            delta = val - old_mean #The difference between the new forecast value and the old mean 
            new_mean = old_mean + (delta /n) #Formula used in Welford's algorithm to update the mean 
            delta2 = val - new_mean #Difference between the new forecast value and the new mean 
            new_var = ((1-(1/n))*old_var) + ((delta*delta2)/n) #Update the value of the sample variance using the population variance formula      
            updated_stats = tf.stack([new_mean, new_var], axis=-1) #Combines the new values of the sample mean and variance 
            current_stats = tf.tensor_scatter_nd_update(current_stats, [[idx]], [updated_stats]) #Overwrites the old values of the mean and variance for a given time series with the new values of the mean and variance 
            return i+1, current_stats, current_count #Set i to i+1 to go to the next time series in the batch, and return the current values of the sample mean and variance, and the number of times the time series have been processed
        i0 = tf.constant(0) #Starting index of the loop 
        stats0 = self.stats #The initial stats for the start of the while loop 
        count0 = self.count #The initial counts that each time series has been processed for the start of the while loop 
        _, updated_stats, updated_count = tf.while_loop( #The while loop which outputs the index, the updated stats, and the updated counts  
            cond=lambda i, *_: i < batch_size, #Determines if the while loop should continue or not, where the loop stops if i exceeds the batch size as there are no more time series to process in the current batch  
            body=loop_body, #Using the function defined earlier which is used in each iteration of the while loop 
            loop_vars=[i0, stats0, count0] #Initial values of the variables in the while loop
        )
        self.stats.assign(updated_stats) #Permanently updates the stats variable of the layer which will be used in future calls of the layer
        self.count.assign(updated_count) #Permanently updates the count variable for the layer which will be used in future calls of the layer 
        out_stats_for_batch = tf.gather(self.stats, series_ids) #Takes the means and variances for the forecasted time points for the time series in the current batch so they can be passed to the proceeding layer 
        return out_stats_for_batch #Returns the means and variances for the forecasted time series in the current batch so they can be passed into the next layer of the network in the current forward pass
class BoltzmannConv1D(Layer): #Defining the Boltzmann convolutional layer (section 4.1) as a subclass of Layer in Keras
    def __init__(self, out_channels, kernel_sizes, dilation_rate=1, padding="causal", temperature=1, **kwargs):#Contructor of the class which takes in several hyperparameters including the type of padding, dilation rate, and temperature of the Boltzmann distribution
        super(BoltzmannConv1D, self).__init__(**kwargs) #Calls the Keras layer class
        self.out_channels = out_channels #Number of output channel
        self.kernel_sizes = kernel_sizes #The set of kernel filter lengths 
        self.dilation_rate = dilation_rate #Dilation rate of the layer
        self.padding = padding #Type of padding for the layer
        self.temperature = temperature #Value of the temperature for the Boltzmann distribution 
        self.convs = [  #Creates a list consisting of sub-convolutional layers with the parameters defined above. The layer is wrapped in a Spatial Concrete Dropout layer 
            ConcreteSpatialDropout1D(
                tf.keras.layers.Conv1D(
                    out_channels, ks,
                    dilation_rate=dilation_rate,
                    padding=padding
                ),
                is_mc_dropout=True,
                weight_regularizer=wr,
                dropout_regularizer=dr
            )
            for ks in kernel_sizes #Iterates over the filter lengths in the kernel_sizes list creating associated convolutional layers
        ]
    def build(self, input_shape): #Create the weights used in the layers
        self.probs = self.add_weight(   #Logits used in the Boltzmann distribution  which are learnable 
            shape=(len(self.kernel_sizes),),
            initializer="uniform",
            trainable=True,
            name="kernel_probs"
        )
        super(BoltzmannConv1D, self).build(input_shape) #Finalizes the build methods by inheriting the build from the overall Keras layer class 
    def call(self, x, training=False): #Main method used when calling the Boltzmann layer for a given forward pass
        kernel_probabilities = tf.nn.softmax(self.probs / self.temperature) #Defines the probability discussed in section 4.1 
        conv_outputs = [ #Creates the list of the weighted convolution outputs  
            conv(x) * kernel_prob
            for conv, kernel_prob in zip(self.convs, tf.unstack(kernel_probabilities)) #Turns the kernel probabilities into a list which are then paired with their associated convolutional layer, with them being multiplied together 
        ]
        out = tf.add_n(conv_outputs) #Sums up the weighted convolutional outputs in the list
        return out #Returns the summed output of the weighted convolutional layers according to (2) in section 4.1
class PredefinedBoltzmannConv1D(Layer):
    def __init__(self, filters, kernel_sizes, dilation_rate, custom_filters, temperature=1, **kwargs):
        super(PredefinedBoltzmannConv1D, self).__init__(**kwargs)
        self.filters = filters #Number of filters 
        self.kernel_sizes = kernel_sizes #The set of kernel filter lengths
        self.temperature = temperature #Value of the temperature for the Boltzmann distribution 
        self.custom_filters = custom_filters #Pre-defined filters as described in section 5
        self.dilation_rate = dilation_rate #Dilation rate of the layer 
        self.convs = []
        for ks in kernel_sizes:
            conv_layer = tf.keras.layers.Conv1D(
                filters=filters,
                kernel_size=ks,
                padding="causal",
                use_bias=False,
                dilation_rate=dilation_rate
            )
            self.convs.append(conv_layer) #Creates a list consisting of sub-convolutional layers with hand-crafted filters with different filter lengths
    def build(self, input_shape):
        for i, (conv_layer, ks) in enumerate(zip(self.convs, self.kernel_sizes)): #Create the weights used in the layers 
            conv_layer.build(input_shape)
            filter_weights = self.custom_filters[ks]
            conv_layer.set_weights([filter_weights])
            conv_layer.trainable = False
        self.probs = self.add_weight( #Logits used in the Boltzmann distribution  which are learnable
            shape=(len(self.kernel_sizes),),
            initializer="uniform",
            trainable=True,
            name="kernel_probs"
        )
        super(PredefinedBoltzmannConv1D, self).build(input_shape)
    def call(self, x, training=False): #Main method used when calling the hand-crafted Boltzmann layer for a given forward pass
        kernel_probabilities = tf.nn.softmax(self.probs / self.temperature)
        conv_outputs = [
            conv(x) * kernel_prob
            for conv, kernel_prob in zip(self.convs, tf.unstack(kernel_probabilities))
        ]
        out = tf.add_n(conv_outputs)
        return out

In [12]:
kernel_sizes = [3,6,12,18,24,30,36] #Pre-defined filter lengths (section 5)
custom_filters_dict = {ks: create_custom_filters(ks) for ks in kernel_sizes} #Pre-defined filter weights as a function of filter length (4)-(6) in section 5
ts_input = Input(shape=(time_steps, 1), name="ts_input")   #Defining the input time series to ForeClassNet           
series_id_input = Input(shape=(), dtype=tf.int32, name="series_id")    #Defining the input time series index to ForeClassNet

#Parallel layer and concatenation layer#
conv_predefined = PredefinedBoltzmannConv1D(filters=3, kernel_sizes=kernel_sizes,dilation_rate=1, custom_filters=custom_filters_dict)
x1 = conv_predefined(ts_input)
x1 = Activation("relu", name="activation_predefined")(x1)
x2 = BoltzmannConv1D(out_channels=128, kernel_sizes=kernel_sizes, dilation_rate=1)(ts_input)
x2 = Activation("relu", name="activation_learnable")(x2)
concatenated = Concatenate(axis=-1)([x1, x2])

#Boltzmann Convolutional layer#
x3 = BoltzmannConv1D(out_channels=64, kernel_sizes=kernel_sizes, dilation_rate=2)(concatenated)
x3 = Activation("relu", name="activation_2")(x3)

#Boltzmann Convolutional layer#
x4 = BoltzmannConv1D(out_channels=32, kernel_sizes=kernel_sizes, dilation_rate=4)(x3)
x4 = Activation("relu", name="activation_3")(x4)

#Time distributed dense layer#
layer_4 = Dense(1, use_bias=False)
layer_4 = ConcreteDenseDropout(layer_4, is_mc_dropout=True, weight_regularizer=wr, dropout_regularizer=dr)
x5 = TimeDistributed(layer_4)(x4)  
x5 = Flatten()(x5)                 
x5_expanded = Lambda(lambda x: tf.expand_dims(x, axis=-1))(x5)
zeros_tensor = Lambda(lambda x: tf.zeros_like(x))(x5_expanded)
x6_final = Concatenate(axis=-1)([x5_expanded, zeros_tensor])

#Fully connected layer#
layer_6 = Dense(32, use_bias=True)  
layer_6 = ConcreteDenseDropout(layer_6, is_mc_dropout=True,weight_regularizer=wr, dropout_regularizer=dr)
x8 = layer_6(x5)
x8 = LeakyReLU(negative_slope=0.01)(x8)

#Fully connected layer and forecast output#
layer_7 = Dense(n_forecast, use_bias=True)
layer_7 = ConcreteDenseDropout(layer_7, is_mc_dropout=True, weight_regularizer=wr, dropout_regularizer=dr,name="output_forecast")
output_forecast = layer_7(x8)

#Welford layer and concatenation layer#
num_series = 90000
welford_forecast_layer = WelfordMeanVarianceLayer(total_series=num_series,n_forecast= n_forecast , name="Welford")
output_forecast_final = welford_forecast_layer([output_forecast, series_id_input])
concatenated_2 = Concatenate(axis=1)([x6_final, output_forecast_final])

#Boltzmann convolutional layer and GAP layer#
x_classification = BoltzmannConv1D(out_channels=32, kernel_sizes=kernel_sizes,dilation_rate=8)(concatenated_2)
x_classification = Activation("relu", name="activation_4")(x_classification)
x_classification = GlobalAveragePooling1D()(x_classification)

#Fully connected layer and classification output#
final_classification_layer = Dense(n_classes, use_bias=False)
final_classification_layer = ConcreteDenseDropout(final_classification_layer, is_mc_dropout=True, weight_regularizer=wr,
                                                  dropout_regularizer=dr,
                                                  name="classification_output")
output_classification = final_classification_layer(x_classification)
output_classification = Activation("softmax")(output_classification)

#compilation and summary of ForeClassNet#
model1 = Model(inputs=[ts_input, series_id_input],outputs=[output_forecast, output_classification]
              ,name="time_series_forecasting_classification_model")
model1.compile(optimizer="adam",loss=["mse", "categorical_crossentropy"],metrics=["mse", "accuracy"])
model1.summary()

Model: "time_series_forecasting_classification_model"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ ts_input            │ (None, 39, 1)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ predefined_boltzma… │ (None, 39, 3)     │        394 │ ts_input[0][0]    │
│ (PredefinedBoltzma… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ boltzmann_conv1d    │ (None, 39, 128)   │     17,422 │ ts_input[0][0]    │
│ (BoltzmannConv1D)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_predefi… │ (None, 39, 3)     │          0 │ predefined_boltz… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_learnab… │ (None, 39, 128)   │          0 │ boltzmann_conv1d… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 39, 131)   │          0 │ activation_prede… │
│ (Concatenate)       │                   │            │ activation_learn… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ boltzmann_conv1d_1  │ (None, 39, 64)    │  1,081,998 │ concatenate[0][0] │
│ (BoltzmannConv1D)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 39, 64)    │          0 │ boltzmann_conv1d… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ boltzmann_conv1d_2  │ (None, 39, 32)    │    264,430 │ activation_2[0][… │
│ (BoltzmannConv1D)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 39, 32)    │          0 │ boltzmann_conv1d… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed    │ (None, 39, 1)     │         33 │ activation_3[0][… │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 39)        │          0 │ time_distributed… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concrete_dense_dro… │ (None, 32)        │      1,281 │ flatten[0][0]     │
│ (ConcreteDenseDrop… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu         │ (None, 32)        │          0 │ concrete_dense_d… │
│ (LeakyReLU)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda (Lambda)     │ (None, 39, 1)     │          0 │ flatten[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output_forecast     │ (None, 1)         │         34 │ leaky_re_lu[0][0] │
│ (ConcreteDenseDrop… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_1 (Lambda)   │ (None, 39, 1)     │          0 │ lambda[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ series_id           │ (None)            │          0 │ -               

 Total params: 1,644,183 (6.27 MB)

 Trainable params: 1,373,796 (5.24 MB)

 Non-trainable params: 270,387 (1.03 MB)

In [13]:
early_stopping = EarlyStopping(monitor='val_activation_accuracy', patience=10,restore_best_weights=True,verbose=1, mode='max')
series_id_train = np.arange(Ns, dtype=np.int32)
batch_size = 128
epochs = 100
tic = time.perf_counter()
history = model1.fit( x = [X_observed_train,series_id_train],y = [X_forecast_train,y_train],epochs=epochs,batch_size=batch_size,
                    validation_split = 0.1,callbacks=[early_stopping], shuffle = True)
toc = time.perf_counter()
print(f"Time taken for fitting: {toc - tic:.2f} seconds")
time_in_mins = round((toc - tic) / 60)
print("Time in minutes:", time_in_mins)

Epoch 1/100
300/300 ━━━━━━━━━━━━━━━━━━━━ 78s 228ms/step - activation_accuracy: 0.3334 - activation_loss: 1.0987 - loss: 1.1372 - output_forecast_loss: 0.0823 - output_forecast_mse: 0.0823 - val_activation_accuracy: 0.3304 - val_activation_loss: 1.0986 - val_loss: 1.0595 - val_output_forecast_loss: 0.0107 - val_output_forecast_mse: 0.0107
Epoch 2/100
300/300 ━━━━━━━━━━━━━━━━━━━━ 69s 230ms/step - activation_accuracy: 0.3338 - activation_loss: 1.0986 - loss: 1.1288 - output_forecast_loss: 0.0822 - output_forecast_mse: 0.0822 - val_activation_accuracy: 0.3287 - val_activation_loss: 1.0986 - val_loss: 1.0506 - val_output_forecast_loss: 0.0107 - val_output_forecast_mse: 0.0107
Epoch 3/100
300/300 ━━━━━━━━━━━━━━━━━━━━ 68s 227ms/step - activation_accuracy: 0.3307 - activation_loss: 1.0986 - loss: 1.1200 - output_forecast_loss: 0.0822 - output_forecast_mse: 0.0822 - val_activation_accuracy: 0.3290 - val_activation_loss: 1.0986 - val_loss: 1.0419 - val_output_forecast_loss: 0.0107 - val_output_f

In [14]:
n_test_series = X_observed_test.shape[0]
test_series_ids = np.arange(n_test_series, dtype=np.int32)
welford_forecast_layer = model1.get_layer("Welford") 
welford_forecast_layer.reset_statistics() #reset the statistics in the Welford layer

In [15]:
def mc_dropout_predict(model, X_test, test_ids, num_samples=100):
    """
    Peforms Monte Carlo dropout, where dropout is applied at inference time. 
    This takes as input the trained ForeClassNet model, the input observed time series, 
    the id's associated with the time series in the test set, and the number of MC samples which
    is the number of forward passes through the trained ForeClassNet model.
    """
    forecast_predictions = [] #List to store the forecast output from each forward pass through ForeClassNet
    classification_predictions = [] #List to store the classification output from each forward pass through ForeClassNet
    for _ in tqdm(range(num_samples), desc="MC Dropout passes"):
        forecast, classification = model.predict([X_test, test_ids], verbose=0) #Performs a forward pass through the network  
        forecast_predictions.append(forecast) #Add the forecast array to the list 
        classification_predictions.append(classification) #Add the classification array of probabilities to the list
    forecast_predictions = np.array(forecast_predictions)
    classification_predictions = np.array(classification_predictions) 
    mean_forecast = np.mean(forecast_predictions, axis=0) #Take the mean with respect to the MC samples of the forecast output
    std_forecast = np.std(forecast_predictions, axis=0) #Take the standard deviation with respect to the MC samples of the forecast output
    mean_classification = np.mean(classification_predictions, axis=0) #Take the mean with respect to the MC samples of the classification probabilities
    std_classification = np.std(classification_predictions, axis=0) #Take the standard deviation with respect to the MC samples of the classification probabilities
    return (mean_forecast, std_forecast), (mean_classification, std_classification)

In [16]:
# Perform MC Dropout predictions with your test data and test IDs
(mean_forecast, forecast_uncertainty), (mean_classification, classification_uncertainty) = mc_dropout_predict(
    model1,X_observed_test,    test_series_ids,    num_samples=100) #Perfrom 100 Monte Carlo passes through the trained ForeClassNet
predicted_classes = np.argmax(mean_classification, axis=-1) #Obtain the predicted classes 
print("Predicted classes:", predicted_classes)
classification_accuracy = accuracy_score(np.argmax(y_test, axis=1), predicted_classes) 
print(classification_accuracy) #Prints the classification accuracy of ForeClassNet on the test set
classification__report = classification_report(np.argmax(y_test, axis=1),predicted_classes, digits = 3) #Evaluate the trained ForeClassNet on the test set in terms of classification accuracy and F1 scores
print(classification__report)

MC Dropout passes: 100%|██████████████████████| 100/100 [16:38<00:00,  9.99s/it]

Predicted classes: [1 0 1 ... 0 2 1]
0.4269346356123216
              precision    recall  f1-score   support

           0      0.449     0.609     0.517      3561
           1      0.388     0.354     0.370      3560
           2      0.434     0.317     0.366      3527

    accuracy                          0.427     10648
   macro avg      0.424     0.427     0.418     10648
weighted avg      0.424     0.427     0.418     10648



In [17]:
actual    = X_forecast_test[:, 0, 0]
last_obs  = X_observed_test[:, -1, 0]
naive_sq_err = (last_obs - actual)**2
naive_mse     = np.mean(naive_sq_err)
model_sq_err = (mean_forecast[:,0] - actual)**2
model_mse     = np.mean(model_sq_err)
print("Naive (persistence) MSE:", naive_mse)
print("Model MSE:              ", model_mse)
print("Improvement:            ",100*(naive_mse - model_mse)/naive_mse, "%")


Naive (persistence) MSE: 0.04535509675229198
Model MSE:               0.015169732364295254
Improvement:             66.55341196349964 %


In [18]:
#Inception time, train multiple inception time models for an increasing number of epochs and using early stopping
X_train_sub, X_val, y_train_sub, y_val = train_test_split(X_observed_train, y_train, test_size=0.1, random_state=42)
patience = 10
best_val_acc = 0.0
no_improve_count = 0
best_model = None
best_epoch_count = 0
max_epochs_to_try = 100
tic = time.perf_counter()
for epoch_count in range(1, max_epochs_to_try + 1):
    print(f"\nAttempting to train a fresh model for {epoch_count} epoch(s)...")
    while True:
        try:
            clf = BaggingClassifier(InceptionTimeClassifier(n_epochs=epoch_count, batch_size=128),n_estimators=5,bootstrap=False)
            clf.fit(X_train_sub.transpose(0, 2, 1), np.argmax(y_train_sub, axis=1))
            break
        except Exception as e:
            print(f"Encountered an error on epoch_count={epoch_count}: {e}")
            print("Retrying the same epoch_count again...")
    val_preds = clf.predict(X_val.transpose(0, 2, 1))
    val_acc = accuracy_score(np.argmax(y_val, axis=1), val_preds)
    print(f"Validation Accuracy after {epoch_count} epoch(s): {val_acc:.4f}")
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_model = clf
        best_epoch_count = epoch_count
        no_improve_count = 0
        print("New best model")
    else:
        no_improve_count += 1
        print(f"No improvement. {no_improve_count} consecutive runs without improvement.")
    if no_improve_count >= patience:
        print(f"\nNo improvement for {patience} consecutive runs. Stopping early.")
        break
toc = time.perf_counter()
print(f"\nBest validation accuracy: {best_val_acc:.4f} (achieved at {best_epoch_count} epoch(s))")
print(f"Time taken for fitting: {toc - tic:.2f} seconds")
if best_model is not None:
    bestpredictions1 = best_model.predict(X_observed_test.transpose(0, 2, 1))
    classification_accuracy = accuracy_score(np.argmax(y_test, axis=1), bestpredictions1)
    print(f"Classification Test Accuracy: {classification_accuracy:.4f}")
    classification_report_str = classification_report(np.argmax(y_test, axis=1),bestpredictions1,digits=3)
    print("Classification Report on Test Set:\n", classification_report_str)
else:
    print("No best model was found (check if training never improved).")


Attempting to train a fresh model for 1 epoch(s)...
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step
Validation Accuracy after 1 epoch(s): 0.3433
New best model

Attempting to train a fresh model for 2 epoch(s)...
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step
Validation Accuracy after 2 epoch(s): 0.3973
New best model

Attempting to train a fresh model for 3 epoch(s)...
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step
Validation Accuracy after 3 epoch(s): 0.4250
New best model

Attempting to train a fresh model for 4 epoch(s)...
34/34 ━━

In [23]:
tf.keras.backend.clear_session()

In [24]:
kernel_sizes = [3,6,12,18,24,30,36] #Pre-defined filter lengths (section 5)
custom_filters_dict = {ks: create_custom_filters(ks) for ks in kernel_sizes} #Pre-defined filter weights as a function of filter length (4)-(6) in section 5
ts_input = Input(shape=(time_steps, 1), name="ts_input")   #Defining the input time series to ForeClassNet      
series_id_input = Input(shape=(), dtype=tf.int32, name="series_id")   #Defining the input time series index to ForeClassNet
 
#Parallel layer and concatenation layer#
conv_predefined = PredefinedBoltzmannConv1D(filters=3, kernel_sizes=kernel_sizes,dilation_rate=1, custom_filters=custom_filters_dict)
x1 = conv_predefined(ts_input)
x1 = Activation("relu", name="activation_predefined")(x1)
x2 = BoltzmannConv1D(out_channels=128, kernel_sizes=kernel_sizes, dilation_rate=1)(ts_input)
x2 = Activation("relu", name="activation_learnable")(x2)
concatenated = Concatenate(axis=-1)([x1, x2])

#Boltzmann Convolutional layer#
x3 = BoltzmannConv1D(out_channels=64, kernel_sizes=kernel_sizes, dilation_rate=2)(concatenated)
x3 = Activation("relu", name="activation_2")(x3)

#Boltzmann convolutional layer#
x4 = BoltzmannConv1D(out_channels=32, kernel_sizes=kernel_sizes, dilation_rate=4)(x3)
x4 = Activation("relu", name="activation_3")(x4)

#Time distributed dense layer#
layer_4 = Dense(1, use_bias=False)
layer_4 = ConcreteDenseDropout(layer_4, is_mc_dropout=True, weight_regularizer=wr, dropout_regularizer=dr)
x5 = TimeDistributed(layer_4)(x4)  
x5 = Flatten()(x5)                 
x5_expanded = Lambda(lambda x: tf.expand_dims(x, axis=-1))(x5)
zeros_tensor = Lambda(lambda x: tf.zeros_like(x))(x5_expanded)
x6_final = Concatenate(axis=-1)([x5_expanded, zeros_tensor])

#Fully connected layer#
layer_6 = Dense(32, use_bias=True)  
layer_6 = ConcreteDenseDropout(layer_6, is_mc_dropout=True,weight_regularizer=wr, dropout_regularizer=dr)
x8 = layer_6(x5)
x8 = LeakyReLU(negative_slope=0.01)(x8)

#Fully connected layer and forecast output#
layer_7 = Dense(n_forecast, use_bias=True)
layer_7 = ConcreteDenseDropout(layer_7, is_mc_dropout=True, weight_regularizer=wr, dropout_regularizer=dr,name="output_forecast")
output_forecast = layer_7(x8)

#Welford layer and concatenation layer#
num_series = 90000
welford_forecast_layer = WelfordMeanVarianceLayer(total_series=num_series,n_forecast= n_forecast , name="Welford")
output_forecast_final = welford_forecast_layer([output_forecast, series_id_input])


#Boltzmann convolutional layer and GAP layer#
x_classification = BoltzmannConv1D(out_channels=32, kernel_sizes=kernel_sizes,dilation_rate=8)(output_forecast_final)
x_classification = Activation("relu", name="activation_4")(x_classification)
x_classification = GlobalAveragePooling1D()(x_classification)

#Fully connected layer and classification output#
final_classification_layer = Dense(n_classes, use_bias=False)
final_classification_layer = ConcreteDenseDropout(final_classification_layer, is_mc_dropout=True, weight_regularizer=wr,
                                                  dropout_regularizer=dr,
                                                  name="classification_output")
output_classification = final_classification_layer(x_classification)
output_classification = Activation("softmax")(output_classification)

#compilation and summary of ForeClassNet#
model2 = Model(inputs=[ts_input, series_id_input],outputs=[output_forecast, output_classification]
              ,name="time_series_forecasting_classification_model")
model2.compile(optimizer="adam",loss=["mse", "categorical_crossentropy"],metrics=["mse", "accuracy"])
model2.summary()

Model: "time_series_forecasting_classification_model"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ ts_input            │ (None, 39, 1)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ predefined_boltzma… │ (None, 39, 3)     │        394 │ ts_input[0][0]    │
│ (PredefinedBoltzma… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ boltzmann_conv1d    │ (None, 39, 128)   │     17,422 │ ts_input[0][0]    │
│ (BoltzmannConv1D)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_predefi… │ (None, 39, 3)     │          0 │ predefined_boltz… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_learnab… │ (None, 39, 128)   │          0 │ boltzmann_conv1d… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 39, 131)   │          0 │ activation_prede… │
│ (Concatenate)       │                   │            │ activation_learn… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ boltzmann_conv1d_1  │ (None, 39, 64)    │  1,081,998 │ concatenate[0][0] │
│ (BoltzmannConv1D)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 39, 64)    │          0 │ boltzmann_conv1d… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ boltzmann_conv1d_2  │ (None, 39, 32)    │    264,430 │ activation_2[0][… │
│ (BoltzmannConv1D)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 39, 32)    │          0 │ boltzmann_conv1d… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed    │ (None, 39, 1)     │         33 │ activation_3[0][… │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 39)        │          0 │ time_distributed… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concrete_dense_dro… │ (None, 32)        │      1,281 │ flatten[0][0]     │
│ (ConcreteDenseDrop… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu         │ (None, 32)        │          0 │ concrete_dense_d… │
│ (LeakyReLU)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output_forecast     │ (None, 1)         │         34 │ leaky_re_lu[0][0] │
│ (ConcreteDenseDrop… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ series_id           │ (None)            │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Welford             │ (None, 1, 2)      │    270,000 │ output_forecast[… │
│ (WelfordMeanVarian… │                   │            │ series_id[0][0] 

 Total params: 1,644,183 (6.27 MB)

 Trainable params: 1,373,796 (5.24 MB)

 Non-trainable params: 270,387 (1.03 MB)

In [25]:
early_stopping = EarlyStopping(monitor='val_activation_accuracy', patience=10,restore_best_weights=True,verbose=1, mode='max')
series_id_train = np.arange(Ns, dtype=np.int32)
batch_size = 128
epochs = 100
tic = time.perf_counter()
history = model2.fit( x = [X_observed_train,series_id_train],y = [X_forecast_train,y_train],epochs=epochs,batch_size=batch_size,
                    validation_split = 0.1,callbacks=[early_stopping], shuffle = True)
toc = time.perf_counter()
print(f"Time taken for fitting: {toc - tic:.2f} seconds")
time_in_mins = round((toc - tic) / 60)
print("Time in minutes:", time_in_mins)

Epoch 1/100
300/300 ━━━━━━━━━━━━━━━━━━━━ 69s 194ms/step - activation_accuracy: 0.3381 - activation_loss: 1.0987 - loss: 1.1372 - output_forecast_loss: 0.0822 - output_forecast_mse: 0.0822 - val_activation_accuracy: 0.3351 - val_activation_loss: 1.0986 - val_loss: 1.0594 - val_output_forecast_loss: 0.0107 - val_output_forecast_mse: 0.0107
Epoch 2/100
300/300 ━━━━━━━━━━━━━━━━━━━━ 58s 193ms/step - activation_accuracy: 0.3326 - activation_loss: 1.0986 - loss: 1.1288 - output_forecast_loss: 0.0822 - output_forecast_mse: 0.0822 - val_activation_accuracy: 0.3325 - val_activation_loss: 1.0986 - val_loss: 1.0506 - val_output_forecast_loss: 0.0107 - val_output_forecast_mse: 0.0107
Epoch 3/100
300/300 ━━━━━━━━━━━━━━━━━━━━ 58s 193ms/step - activation_accuracy: 0.3286 - activation_loss: 1.0987 - loss: 1.1200 - output_forecast_loss: 0.0822 - output_forecast_mse: 0.0822 - val_activation_accuracy: 0.3231 - val_activation_loss: 1.0987 - val_loss: 1.0420 - val_output_forecast_loss: 0.0107 - val_output_f

In [26]:
n_test_series = X_observed_test.shape[0]
test_series_ids = np.arange(n_test_series, dtype=np.int32)
welford_forecast_layer = model2.get_layer("Welford") 
welford_forecast_layer.reset_statistics() #reset the statistics in the Welford layer

In [27]:
# Perform MC Dropout
(mean_forecast, forecast_uncertainty), (mean_classification, classification_uncertainty) = mc_dropout_predict(
    model2,X_observed_test,    test_series_ids,    num_samples=100) #Perfrom 100 Monte Carlo passes through the trained ForeClassNet
predicted_classes2 = np.argmax(mean_classification, axis=-1) #Obtain the predicted classes 
print("Predicted classes:", predicted_classes2)
classification_accuracy = accuracy_score(np.argmax(y_test, axis=1), predicted_classes2) 
print(classification_accuracy) #Prints the classification accuracy of ForeClassNet on the test set
classification__report = classification_report(np.argmax(y_test, axis=1),predicted_classes2,digits = 3) #Evaluate the trained ForeClassNet on the test set in terms of classification accuracy and F1 scores
print(classification__report)

MC Dropout passes: 100%|██████████████████████| 100/100 [13:17<00:00,  7.98s/it]

Predicted classes: [2 2 2 ... 2 2 2]
0.3312359128474831
              precision    recall  f1-score   support

           0      0.000     0.000     0.000      3561
           1      0.000     0.000     0.000      3560
           2      0.331     1.000     0.498      3527

    accuracy                          0.331     10648
   macro avg      0.110     0.333     0.166     10648
weighted avg      0.110     0.331     0.165     10648




/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [22]:
a = b = c = d = 0 #Initialize variables to store counts for the contingency table used in McNemar's test
for i in range(len(y_test)): #Loop through the number of examples in the test set 
    correct_1 = (bestpredictions1[i] == np.argmax(y_test[i])) #Returns 1 if Inception time gives a correct prediction and 0 otherwise
    correct_2 = (predicted_classes[i] == np.argmax(y_test[i])) #Returns 1 if ForeClassNet gives a correct prediction and 0 otherwise
    
    if  correct_1 and  correct_2:
        a += 1 #Both classifiers gave the correct prediction
    elif correct_1 and not correct_2:
        b += 1 #Inception time gave the correct prediction and ForeClassNet did not
    elif not correct_1 and correct_2:
        c += 1 #ForeClassNet gave the correct prediction and Inception time did not
    else:
        d += 1 #Both classifiers gave the incorrect prediction

table = [[a, b],
         [c, d]] #Construct contingency table 
print("Contingency table:")
print(np.array(table))
result = mcnemar(table, exact=False, correction=True) #Perform McNemar's test 
print("statistic:", result.statistic) #Return the test statistic from McNemar's test
print("p-value:  ", result.pvalue) #Return the p-value from McNemar's test

Contingency table:
[[3697  939]
 [ 849 5163]]
statistic: 4.430089485458613
p-value:   0.035310656600195334
